In [ ]:
import os
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from matplotlib import pyplot as plt

s3 = S3AccessUtils('/root/data')
RDS = RDSAccessUtils()

In [ ]:
def extract_biomass_data(pen_id, start_date, end_date, akpd_score_cutoff):
    """Get raw biomass computations for given pen_id, date range, and AKPD score cutoff."""

    query = """
        SELECT * FROM
        prod.biomass_computations bc
        WHERE bc.pen_id={}
        AND bc.akpd_score >= {}
        AND bc.captured_at BETWEEN '{}' and '{}'
        AND bc.estimated_weight_g > 0.0
    """.format(pen_id, akpd_score_cutoff, start_date, end_date)

    df = RDS.extract_from_database(query)
    return df

In [ ]:
pen_id = 145
start_date = '2021-02-06'
end_date = '2021-02-10'
akpd_score_cutoff = 0.95
df = extract_biomass_data(pen_id, start_date, end_date, akpd_score_cutoff)

In [ ]:
import json
from typing import Dict
import cv2
import numpy as np

IMAGE_WIDTH = 4096
IMAGE_HEIGHT = 3000


def get_camera_parameters(params: Dict) -> Dict:
    """Return individual camera parameters from JSON stereo parameters contents."""

    camera_matrix_1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    camera_matrix_2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    dist_coeffs_1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                    params['CameraParameters1']['TangentialDistortion'] + \
                    [params['CameraParameters1']['RadialDistortion'][2]]
    dist_coeffs_1 = np.array(dist_coeffs_1)

    dist_coeffs_2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                    params['CameraParameters2']['TangentialDistortion'] + \
                    [params['CameraParameters2']['RadialDistortion'][2]]
    dist_coeffs_2 = np.array(dist_coeffs_2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(camera_matrix_1, dist_coeffs_1,
                                                               camera_matrix_2,
                                                               dist_coeffs_2, image_size, R, T,
                                                               None,
                                                               None,
                                                               None, None, None,
                                                               cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(camera_matrix_1, dist_coeffs_1, R1, P1, image_size,
                                            cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(camera_matrix_2, dist_coeffs_2, R2, P2, image_size,
                                             cv2.CV_16SC2)

    params = {
        'left_maps': left_maps,
        'right_maps': right_maps,
        'camera_matrix_1': camera_matrix_1,
        'dist_coeffs_1': dist_coeffs_1,
        'R1': R1,
        'P1': P1,
        'camera_matrix_2': camera_matrix_2,
        'dist_coeffs_2': dist_coeffs_2,
        'R2': R2,
        'P2': P2
    }
    return params


def unrectify(ann: Dict, params: Dict):
    """Un-rectify ann with params."""

    left_maps = params['left_maps']
    right_maps = params['right_maps']

    ann_u = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = left_maps[0][y, x]
            elif side == 'rightCrop':
                x_new, y_new = right_maps[0][y, x]
            else:
                raise Exception('Invalid side!')

            ann_u[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_u


def rectify(ann: Dict, params: Dict) -> Dict:
    """Rectify ann with params."""

    camera_matrix_1 = params['camera_matrix_1']
    dist_coeffs_1 = params['dist_coeffs_1']
    R1 = params['R1']
    P1 = params['P1']

    camera_matrix_2 = params['camera_matrix_2']
    dist_coeffs_2 = params['dist_coeffs_2']
    R2 = params['R2']
    P2 = params['P2']

    ann_r = {'leftCrop': [], 'rightCrop': []}
    for side in ['leftCrop', 'rightCrop']:
        for item in ann[side]:
            bp = item['keypointType']
            x = item['xFrame']
            y = item['yFrame']
            if side == 'leftCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[x, y]]).astype(float),
                        camera_matrix_1,
                        dist_coeffs_1,
                        R=R1,
                        P=P1)[0][0]
            elif side == 'rightCrop':
                x_new, y_new = \
                    cv2.undistortPoints(
                        np.array([[x, y]]).astype(float),
                        camera_matrix_2,
                        dist_coeffs_2,
                        R=R2,
                        P=P2)[0][0]
            else:
                raise Exception('Invalid side!')

            ann_r[side].append({
                'keypointType': bp,
                'xFrame': x_new,
                'yFrame': y_new,
            })

    return ann_r


def un_re_rectify(df, params_o, params_n):

    ann_u_rs = []
    for idx, row in df.iterrows():
        ann = row.annotation
        if ann is None:
            ann_u_rs.append(None)
            continue

        # un-rectify with original params
        ann_u = unrectify(ann, params_o)
        if ann_u:

            # re-rectify with new params
            ann_u_r = rectify(ann_u, params_n)
            ann_u_rs.append(ann_u_r)
        else:
            ann_u_rs.append(None)

    df['ann'] = ann_u_rs

    
def add_camera_metadata(df, stereo_parameters):
    
    camera_metadata = {
        'focalLengthPixel': stereo_parameters['CameraParameters1']['FocalLength'][0],
        'baseline': abs(stereo_parameters['TranslationOfCamera2'][0] / 1e3),
        'focalLength': stereo_parameters['CameraParameters1']['FocalLength'][0] * 3.45e-6,
        'pixelCountWidth': 4096,
        'pixelCountHeight': 3000,
        'imageSensorWidth': 0.01412,
        'imageSensorHeight': 0.01035
    }

    df['camera_metadata'] = json.dumps(camera_metadata)
    
    

def add_weights(df, k):
    
    weight_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb')
    kf_model_f, _, _ = s3.download_from_url('https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb')
    weight_estimator = WeightEstimator(weight_model_f, kf_model_f)

    pred_weights = []
    for idx, row in df.iterrows():
        ann = row.ann
        camera_metadata = json.loads(row.camera_metadata)
        if ann is not None:
            cm = CameraMetadata(
                focal_length=camera_metadata['focalLength'],
                focal_length_pixel=camera_metadata['focalLengthPixel'],
                baseline_m=camera_metadata['baseline'],
                pixel_count_width=camera_metadata['pixelCountWidth'],
                pixel_count_height=camera_metadata['pixelCountHeight'],
                image_sensor_width=camera_metadata['imageSensorWidth'],
                image_sensor_height=camera_metadata['imageSensorHeight']
            )

            weight, _, _ = weight_estimator.predict(ann, cm)
            pred_weights.append(weight)
        else:
            pred_weights.append(None)
    
    df['weight_{}'.format(k)] = pred_weights
    

def get_stereo_parameters(stereo_parameter_url):
    if stereo_parameter_url.startswith('s3://'):
        stereo_parameters_components = stereo_parameter_url.replace('s3://', '').split('/')
        bucket = stereo_parameters_components[0]
        key = os.path.join(*stereo_parameters_components[1:])
        stereo_parameters_f = s3.download_from_s3(bucket, key)
    elif stereo_parameter_url.startswith('https://'):
        stereo_parameters_f, _, _ = s3.download_from_url(stereo_parameter_url)
    else:
        raise Exception('Unrecognized URL format')
    return stereo_parameters_f

In [ ]:
stereo_parameter_urls = [
    ('parameters1', 's3://aquabyte-stereo-parameters/L40076048_R40076044/2021-01-07T12:17:15.547471000Z_L40076048_R40076044_stereo-parameters.json'),
    ('parameters2', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-03-10T13:57:48Z-pfe-1421920048928-187-4bd8/cal_output/2021-03-10T14-07-03.821272000Z/stereo_params.json'),
    ('rot1', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A17%3A33Z-pfe-1421920048928-9f6-46e1/cal_output/2021-04-06T10-32-38.492361000Z/stereo_params.json'),
    ('rot2', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A25%3A01Z-pfe-1421920048928-1b8-4398/cal_output/2021-04-06T10-59-45.370389000Z/stereo_params.json'),
    ('rot3', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A32%3A30Z-pfe-1421920048928-5a9-46c6/cal_output/2021-04-06T10-45-35.798195000Z/stereo_params.json'),
    ('rot4', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A40%3A44Z-pfe-1421920048928-ea1-4697/cal_output/2021-04-06T11-27-09.267188000Z/stereo_params.json'),
    ('rot5', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A48%3A17Z-pfe-1421920048928-dcb-445c/cal_output/2021-04-06T11-15-19.086423000Z/stereo_params.json'),
    ('shake1', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T10%3A57%3A54Z-pfe-1421920048928-675-46e5/cal_output/2021-04-06T11-48-27.629782000Z/stereo_params.json'),
    ('shake2', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T11%3A06%3A52Z-pfe-1421920048928-5db-4c60/cal_output/2021-04-06T12-00-13.274660000Z/stereo_params.json'),
    ('shake3', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T11%3A41%3A00Z-pfe-1421920048928-f86-48bb/cal_output/2021-04-06T12-11-21.222269000Z/stereo_params.json'),
    ('shake4', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T11%3A50%3A04Z-pfe-1421920048928-635-49bf/cal_output/2021-04-06T12-36-00.638934000Z/stereo_params.json'),
    ('shake5', 'https://aquabyte-abc.s3-eu-west-1.amazonaws.com/rook/2021-04-06T12%3A06%3A14Z-pfe-1421920048928-390-424d/cal_output/2021-04-06T12-22-49.207767000Z/stereo_params.json')
]

stereo_parameter_urls = [(x, y.replace('%3A', ':')) for x, y in stereo_parameter_urls]


In [ ]:
stereo_parameter_urls = [(x, y.replace('%3A', ':')) for x, y in stereo_parameter_urls]
stereo_parameters_f = get_stereo_parameters(stereo_parameter_urls[0][1])
stereo_parameters = json.load(open(stereo_parameters_f))
params_o = get_camera_parameters(stereo_parameters)


In [ ]:
for k, stereo_parameter_url in stereo_parameter_urls:
    stereo_parameters_f = get_stereo_parameters(stereo_parameter_url)
    stereo_parameters = json.load(open(stereo_parameters_f))
    params = get_camera_parameters(stereo_parameters)
    un_re_rectify(df, params_o, params)
    add_camera_metadata(df, stereo_parameters)
    add_weights(df, k)
    print(k, df['weight_{}'.format(k)].mean())

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist((df['weight_rot1'] - df['weight_rot2']).values)
plt.grid()
plt.show()